# Creating a Sqlite DB

To avoid retrieving data from dofusDB with a lot of requests each time we need it, we retrieved all important data as json and we build a simple sqlite database

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('dofusdb.sqlite')

In [3]:
df = pd.json_normalize(pd.read_json('data/map-positions.json').data)
df_to_store = df[['id', 'posX', 'posY', 'subAreaId', 'worldMap']].set_index('id')
df_to_store.to_sql('maps', conn, if_exists='replace', index=True)

13673

In [5]:
df = pd.json_normalize(pd.read_json('data/subareas.json').data)
print(df.columns)
to_store = df[['id','name.fr', 'mapIds', 'bounds.x', 'bounds.y', 'bounds.width', 'bounds.height', 'worldmapId']].set_index('id')
to_store['mapIds'] = to_store['mapIds'].astype(str)
to_store.rename({'name.fr':'name'})
to_store.to_sql('subareas', conn, if_exists='replace', index=True)


Index(['_id', 'playlists', 'mapIds', 'shape', 'customWorldMap', 'monsters',
       'entranceMapIds', 'exitMapIds', 'achievements', 'quests', 'questsIds',
       'npcs', 'npcsIds', 'harvestables', 'id', 'nameId', 'areaId',
       'worldmapId', 'packId', 'level', 'isConquestVillage',
       'basicAccountAllowed', 'displayOnWorldMap', 'mountAutoTripAllowed',
       'psiAllowed', 'capturable', 'exploreAchievementId',
       'associatedZaapMapId', 'createdAt', 'updatedAt', '__v', 'name.de',
       'name.en', 'name.es', 'name.fr', 'name.it', 'name.pt', 'name.id',
       'bounds.x', 'bounds.y', 'bounds.width', 'bounds.height'],
      dtype='object')


461

In [6]:
df = pd.json_normalize(pd.read_json('data/subareas.json').data).set_index('id')
df[['name.fr', 'mapIds',  'areaId', 'associatedZaapMapId']].loc[450]

name.fr                                                   Route des âmes
mapIds                 [153879299, 153880835, 153880323, 153356296, 1...
areaId                                                                45
associatedZaapMapId                                                    0
Name: 450, dtype: object

In [7]:
df = pd.json_normalize(pd.read_json('data/areas.json').data)
df_to_store = df[['id', 'name.fr','superAreaId']].set_index('id')
df_to_store.to_sql('areas', conn, if_exists='replace', index=True)

62

In [8]:
df = pd.json_normalize(pd.read_json('data/quests.json').data)
df_to_store = df[['id', 'name.fr', 'startCriterion', 'categoryId']].set_index('id')
df_to_store.to_sql('quests', conn, if_exists='replace', index=True)

1929

In [9]:
def elements_to_text(elements):
    text=''
    if type(elements) != list:
         elements = [elements]
    for el in elements:
            if type(el) == str:
                text += el
            elif type(el) == int:
                text += str(el)
            elif type(el) == dict and "name" in el and 'fr' in el['name']:
                text += f"{el['name']['fr']} (type: {el['type']})"
    return text

In [10]:
df = pd.json_normalize(pd.read_json('data/quest-objectives.json').data)
for param in  ['parameters.parameter0',
'parameters.parameter1',
'parameters.parameter2',
'parameters.parameter3',
'parameters.parameter4'] :
    df[param] = df[param].fillna(-1).astype(int)
    
df['text']=df['text'].apply(elements_to_text)
df_to_store = df[['id', 'typeId', 'text', 'map.subAreaId','stepId', 'parameters.parameter0',
'parameters.parameter1',
'parameters.parameter2',
'parameters.parameter3',
'parameters.parameter4'
]].set_index('id')

df_to_store['subAreaId'] = df_to_store['map.subAreaId'].fillna(-1).astype(int)
df_steps = pd.json_normalize(pd.read_json("data/quest-steps.json").data)
df_steps = df_steps[["id","objectiveIds",  "questId"]].explode('objectiveIds').set_index("objectiveIds")
df_final = pd.concat([df_to_store,df_steps], axis=1)[[ 'typeId', 'text', 'subAreaId','questId', 'parameters.parameter0',
'parameters.parameter1',
'parameters.parameter2',
'parameters.parameter3',
'parameters.parameter4'
]]
df_final.to_sql('objectives', conn, if_exists='replace', index=True)
df_final

,typeId,text,subAreaId,questId,parameters.parameter0,parameters.parameter1,parameters.parameter2,parameters.parameter3,parameters.parameter4
102,4,Découvrir la carte : Laboratoire Wabbit,-1,18,287243,0,0,0,0
104,3,Ramener à Assistant d'Otomaï (type: npcs) : x1...,10,18,119,1746,1,0,0
105,3,Ramener à Wogew l'hewmite (type: npcs) : x1 An...,56,18,196,1747,1,0,0
115,6,Vaincre x1 Wabbit Gm (type: monsters) en un se...,-1,18,182,1,0,0,0
133,4,Découvrir la carte : Souterrain de la Biblioth...,488,26,55374,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18601,0,Choisir un altéré auprès de Profèche Orsaine,-1,2462,954792,0,0,0,0
18602,0,Poursuivre votre aventure en vous adressant à ...,-1,2462,954793,0,0,0,0
18911,4,Découvrir la carte : Intérieur de l'Auberge de...,482,2483,997946,0,0,0,0
18912,1,Aller voir Clionora (type: npcs),482,2483,7037,0,0,0,0


In [11]:
conn.close()